<a href="https://colab.research.google.com/github/Dona134/MDA-Amazon-Book-Reviews/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import ast
import os
import re

## Downloading and Extracting the Dataset

In [2]:
os.environ['KAGGLE_USERNAME'] = "aidanaakkaziyeva"
os.environ['KAGGLE_KEY'] = "609e0e320a0900d9d1865319a498c843"
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 97% 1.04G/1.06G [00:05<00:00, 253MB/s]
100% 1.06G/1.06G [00:05<00:00, 215MB/s]


In [3]:
import zipfile

# Define the zip file name
zip_file = "amazon-books-reviews.zip"

# Extract all contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("amazon-books-reviews")  # Extracts into a folder

In [4]:
os.listdir("amazon-books-reviews")

['Books_rating.csv', 'books_data.csv']

## Loading the Books Data

In [5]:
books_data = pd.read_csv("amazon-books-reviews/books_data.csv")

In [6]:
books_data.head(5)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [7]:
books_data.shape

(212404, 10)

##  Preprocessing the Books Data

In [8]:
# keep only columns 'Title', 'categories'
books_data = books_data[['Title', 'categories', 'authors', 'publishedDate']]
books_data.head(5)

,Title,categories,authors,publishedDate
0,Its Only Art If Its Well Hung!,['Comics & Graphic Novels'],['Julie Strain'],1996
1,Dr. Seuss: American Icon,['Biography & Autobiography'],['Philip Nel'],2005-01-01
2,Wonderful Worship in Smaller Churches,['Religion'],['David R. Ray'],2000
3,Whispers of the Wicked Saints,['Fiction'],['Veronica Haddon'],2005-02
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],2003-03-01


In [9]:
# show unique values in the column categories
books_data['categories'].nunique()

10883

In [10]:
# show top 20 categories
books_data['categories'].value_counts().head(20)

,count
categories,
['Fiction'],23419
['Religion'],9459
['History'],9330
['Juvenile Fiction'],6643
['Biography & Autobiography'],6324
['Business & Economics'],5625
['Computers'],4312
['Social Science'],3834
['Juvenile Nonfiction'],3446


In [11]:
books_data.loc[:, 'categories'] = books_data['categories'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

## Filtering for Cooking Books

In [12]:
# Filter the dataframe to include only rows where the categories column contains 'Cooking' (and not missing)
df_cooking = books_data[
    books_data['categories'].apply(
        lambda x: isinstance(x, list) and any(cat.strip().lower() == 'cooking' for cat in x)
    )
]

In [13]:
df_cooking.head()

,Title,categories,authors,publishedDate
19,Alaska Sourdough,[Cooking],['Ruth Allman'],1976
199,Old-Fashioned Ckbk,[Cooking],['Don Holm'],1969
280,Basil: A Book of Recipes (The Little Recipe Bo...,[Cooking],NaN,1997
351,Vino Para Dummies,[Cooking],"['Ed McCarthy', 'Mary Ewing-Mulligan']",2011-03-03
423,Flavors of Korea: Delicious Vegetarian Cuisine...,[Cooking],"['Deborah Coultrip-Davis', 'Young Sook Ramsay']",1998-01-01


In [14]:
df_cooking.shape

(2452, 4)

In [15]:
# remove rows with the same values for authors and publishedDate
df_cooking = df_cooking.drop_duplicates(subset=['authors', 'publishedDate'])

# remove rows with the same values for title
df_cooking = df_cooking.drop_duplicates(subset=['Title'])

In [16]:
df_cooking.shape

(2319, 4)

In [17]:
# keep only column Title
df_cooking = df_cooking[['Title']]

In [18]:
df_cooking.head()

,Title
19,Alaska Sourdough
199,Old-Fashioned Ckbk
280,Basil: A Book of Recipes (The Little Recipe Bo...
351,Vino Para Dummies
423,Flavors of Korea: Delicious Vegetarian Cuisine...


## Loading and Preprocessing Books Rating Data

In [19]:
books_rating = pd.read_csv("amazon-books-reviews/Books_rating.csv")

In [20]:
books_rating.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [21]:
# drop the columns that are not needed (Price, review/time, review/summary, profileName, review/text, review/helpfulness)
books_rating = books_rating[['Title', 'review/text', 'User_id', 'review/time']]
books_rating.head()

,Title,review/text,User_id,review/time
0,Its Only Art If Its Well Hung!,This is only for Julie Strain fans. It's a col...,AVCGYZL8FQQTD,940636800
1,Dr. Seuss: American Icon,I don't care much for Dr. Seuss but after read...,A30TK6U7DNS82R,1095724800
2,Dr. Seuss: American Icon,"If people become the books they read and if ""t...",A3UH4UZ4RSVO82,1078790400
3,Dr. Seuss: American Icon,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",A2MVUWT453QH61,1090713600
4,Dr. Seuss: American Icon,Philip Nel - Dr. Seuss: American IconThis is b...,A22X4XUPKF66MR,1107993600


In [22]:
# remove rows with the same unique values for User_id and review/time
books_rating = books_rating.drop_duplicates(subset=['User_id', 'review/time']) #unix timestamp
books_rating.shape

(1548183, 4)

In [23]:
# drop the column review/time
books_rating = books_rating.drop(columns=['review/time'])

In [24]:
# display null values in the dataset
books_rating.isnull().sum()

,0
Title,17
review/text,1
User_id,5558


In [25]:
# remove null values
books_rating = books_rating.dropna()

In [26]:
books_rating.shape

(1542607, 3)

## Merging Cooking Books with Ratings

In [27]:
# match the books_rating data with the df_romance data using the Title column

df_cooking = df_cooking.merge(books_rating, on='Title', how='inner')
df_cooking.head()

,Title,review/text,User_id
0,Alaska Sourdough,"I have been using this book since 1988, the ei...",AC58Z72OB2DDX
1,Alaska Sourdough,"My poor dogeared, stained copy of this book ca...",A3CNQIKVTG9QYO
2,Alaska Sourdough,"As a former Alaskan, I didn't want to have to ...",A2UMP9TJTJ6A6B
3,Alaska Sourdough,For those of us who would prefer to use sourdo...,AC2TK7NHKB5C0
4,Alaska Sourdough,Make the most sublime waffles - crispy outside...,A22T74YNRM8NTK


## Final Cleaning

In [28]:
# drop rows where User_id is not all caps and that has spaces
df_cooking = df_cooking[df_cooking['User_id'].str.isupper() & ~df_cooking['User_id'].str.contains(' ')]
df_cooking.shape

(22434, 3)

In [29]:
# null values
df_cooking.isnull().sum()

,0
Title,0
review/text,0
User_id,0


In [30]:
# Count records per book
df_cooking['Title'].value_counts()

,count
Title,
The Bread Lover's Bread Machine Cookbook: A Master Baker's 300 Favorite Recipes for Perfect-Every-Time Bread-From Every Kind of Machine,398
Bread Baker's Apprentice,358
How To Cook Everything: Simple Recipes for Great Food,293
Eating For Life,263
"Saving Dinner: The Menus, Recipes, and Shopping Lists to Bring Your Family Back to the Table",250
...,...
The Co-ed cookbook (Scholastic starline),1
Cooking Vegetables,1
Cooking for Heart and Soul: 100 Delicious Low-Fat Recipes from San Francisco's Top ChefsA Cookbook to Benefit the San Francisco Food Bank,1


In [31]:
# average length of the reviews
df_cooking['review/text'].apply(lambda x: len(str(x).split())).mean()

np.float64(112.57805117232772)

# Data Processing

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace
from pyspark.ml.feature import MinHashLSH, HashingTF
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import lower, regexp_replace

In [33]:
from pyspark.sql import SparkSession

## Initialize Spark Session

In [34]:
spark = SparkSession.builder \
    .appName("BooksReviewSimilarity") \
    .config("spark.driver.memory", "8g") \
    .master("local[*]") \
    .getOrCreate()

## Data Exploration

In [35]:
# load the DataFrame from pandas to Spark
df_cooking = spark.createDataFrame(df_cooking)

In [36]:
df_cooking.show(5)

+----------------+--------------------+--------------+
|           Title|         review/text|       User_id|
+----------------+--------------------+--------------+
|Alaska Sourdough|I have been using...| AC58Z72OB2DDX|
|Alaska Sourdough|My poor dogeared,...|A3CNQIKVTG9QYO|
|Alaska Sourdough|As a former Alask...|A2UMP9TJTJ6A6B|
|Alaska Sourdough|For those of us w...| AC2TK7NHKB5C0|
|Alaska Sourdough|Make the most sub...|A22T74YNRM8NTK|
+----------------+--------------------+--------------+
only showing top 5 rows



In [37]:
df_cooking.count(), len(df_cooking.columns)

(22434, 3)

## Data Subsampling

In [38]:
# subsample the dataset (50%)
# df_cooking = df_cooking.sample(0.5, seed=123)
# df_cooking.count(), len(df_cooking.columns)


total_rows = df_cooking.count()

# Calculate the number of rows for the first half
half_rows = total_rows // 2

# Subset the first half of the dataset
df_cooking = df_cooking.limit(half_rows)

df_cooking.count(), len(df_cooking.columns)

(11217, 3)

In [40]:
# ensure that a user only has one entry per book
df_cooking = df_cooking.dropDuplicates(
    subset=["Title", "User_id"]
)

In [41]:
df_clean = df_cooking.withColumn(
    "clean_text",
    regexp_replace(lower(col("review/text")), r'[^\w\s]', '')
)

In [42]:
df_clean.show(5)

+--------------------+--------------------+--------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|
+--------------------+--------------------+--------------+--------------------+
|1,000 Vegetarian ...|This book is one ...|A12KSAYBWRE2EP|this book is one ...|
|1,000 Vegetarian ...|I have used this ...|A14O63Q1W61YO5|i have used this ...|
|1,000 Vegetarian ...|In our house, we ...|A17FLJJOOO0M9V|in our house we h...|
|1,000 Vegetarian ...|I am pleasantly s...|A1BBJK1VE07KCV|i am pleasantly s...|
|1,000 Vegetarian ...|I own just about ...|A1FMR10ARQHGKF|i own just about ...|
+--------------------+--------------------+--------------+--------------------+
only showing top 5 rows



## Text Preprocessing

In [43]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [44]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, MinHashLSH

In [45]:
tokenizer = Tokenizer(inputCol="clean_text", outputCol="words")
df_tokenized = tokenizer.transform(df_clean)

In [46]:
from pyspark.ml.feature import StopWordsRemover

# Define custom stopwords
custom_stopwords = ["book", "read", "recipe", "good", "great"]

# Combine with default stopwords
default_stopwords = StopWordsRemover.loadDefaultStopWords("english")
all_stopwords = default_stopwords + custom_stopwords

remover = StopWordsRemover(inputCol="words", outputCol="filtered_text", stopWords=all_stopwords)
df_no_stop = remover.transform(df_tokenized)

## Feature Engineering

In [47]:
# Create word-level shingles (n-grams)
def create_word_shingles(words, n=2):
    if len(words) < n:
        return [" ".join(words)]
    return [" ".join(words[i:i+n]) for i in range(len(words) - n + 1)]

# Define UDF
shingle_udf = udf(lambda words: create_word_shingles(words, n=2), ArrayType(StringType()))

# Apply UDF to generate shingles
df_shingled = df_no_stop.withColumn("shingles", shingle_udf(col("filtered_text")))

df_shingled.show(5)

+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|               words|       filtered_text|            shingles|
+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|1,000 Vegetarian ...|This book is one ...|A12KSAYBWRE2EP|this book is one ...|[this, book, is, ...|[one, bibles, veg...|[one bibles, bibl...|
|1,000 Vegetarian ...|I have used this ...|A14O63Q1W61YO5|i have used this ...|[i, have, used, t...|[used, cookbook, ...|[used cookbook, c...|
|1,000 Vegetarian ...|In our house, we ...|A17FLJJOOO0M9V|in our house we h...|[in, our, house, ...|[house, several, ...|[house several, s...|
|1,000 Vegetarian ...|I am pleasantly s...|A1BBJK1VE07KCV|i am pleasantly s...|[i, am, pleasantl...|[pleasantly, supr...|[pleasantly supri...|

In [48]:
# Create feature vectors (using hashing trick)
from pyspark.ml.feature import HashingTF

hashing_tf = HashingTF(inputCol="shingles", outputCol="features", numFeatures=1 << 20)  # 2^20 features
featurized_data = hashing_tf.transform(df_shingled)   #each bigram gets hashed to an index between 0 and 1,048,575 and the value = freq of occurence (sparce vector).

In [49]:
featurized_data.show(5)

+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|               words|       filtered_text|            shingles|            features|
+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1,000 Vegetarian ...|This book is one ...|A12KSAYBWRE2EP|this book is one ...|[this, book, is, ...|[one, bibles, veg...|[one bibles, bibl...|(1048576,[19336,5...|
|1,000 Vegetarian ...|I have used this ...|A14O63Q1W61YO5|i have used this ...|[i, have, used, t...|[used, cookbook, ...|[used cookbook, c...|(1048576,[10768,2...|
|1,000 Vegetarian ...|In our house, we ...|A17FLJJOOO0M9V|in our house we h...|[in, our, house, ...|[house, several, ...|[house several, s...|(1048576,[48330,1...|
|1,000 Vegetaria

In [50]:
from pyspark.sql.functions import monotonically_increasing_id

df_with_id = featurized_data.withColumn("Review_ID", monotonically_increasing_id())

## MinHash LSH for Similarity Search

In [51]:
# Apply MinHash
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=6)
model = mh.fit(df_with_id)

In [52]:
# Find similar reviews
similar_reviews = model.approxSimilarityJoin(
    df_with_id, df_with_id, 0.8, distCol="JaccardDistance"
)

In [53]:
filtered_df = similar_reviews.filter(col("datasetA.Review_ID") != col("datasetB.Review_ID"))

In [54]:
# keeping only one direction of each pair
filtered_df = filtered_df.filter(col("datasetA.review_ID") < col("datasetB.review_ID"))

In [55]:
top3 = (
    filtered_df
    .orderBy(col("JaccardDistance").asc())
    .limit(3)
)

top3.select(
    col("datasetA.Title").alias("Title_A"),
    col("datasetB.Title").alias("Title_B"),
    col("datasetA.review/text").alias("review/text_A"),
    col("datasetB.review/text").alias("review/text_B"),
    col("JaccardDistance")
).show(5, truncate=False)

+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## TF-IDF + Cosine LSH

In [58]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, BucketedRandomProjectionLSH
from pyspark.sql.functions import col

In [59]:
from pyspark.sql.functions import size

# exclude rows where 'filtered_text' has less than 6 words
featurized_data = featurized_data.filter(size(col("filtered_text")) >= 6)

In [60]:
# Apply IDF to get TF-IDF vectors
idf = IDF(inputCol="features", outputCol="tfidf_features")
idf_model = idf.fit(featurized_data)
df_tfidf = idf_model.transform(featurized_data)

In [61]:
df_tfidf.show(5)

+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         review/text|       User_id|          clean_text|               words|       filtered_text|            shingles|            features|      tfidf_features|
+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1,000 Vegetarian ...|This book is one ...|A12KSAYBWRE2EP|this book is one ...|[this, book, is, ...|[one, bibles, veg...|[one bibles, bibl...|(1048576,[19336,5...|(1048576,[19336,5...|
|1,000 Vegetarian ...|I have used this ...|A14O63Q1W61YO5|i have used this ...|[i, have, used, t...|[used, cookbook, ...|[used cookbook, c...|(1048576,[10768,2...|(1048576,[10768,2...|
|1,000 Vegetarian ...|In our house, we ...|A17FLJJOOO0M9V|in our house we h

In [62]:
from pyspark.sql.functions import monotonically_increasing_id

df_with_id = df_tfidf.withColumn("Review_ID", monotonically_increasing_id())

In [63]:
# LSH with Cosine-like projection (BucketedRandomProjectionLSH)
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.5, numHashTables=4)  #number of different “ways” you're looking for similarity.


brp_model = brp.fit(df_with_id)

In [64]:
# Self-join to find similar review pairs
similar_pairs = brp_model.approxSimilarityJoin(df_with_id, df_with_id, threshold=5.0, distCol="distance")

In [65]:
# Filter out duplicates and self-pairs
similar_pairs_filtered = similar_pairs \
    .filter(col("datasetA.Review_ID") < col("datasetB.Review_ID")) \
    .filter(col("datasetA.Review_ID") != col("datasetB.Review_ID")) \
    .select("datasetA.Title", "datasetB.Title", "datasetA.User_id", "datasetB.User_id", "datasetA.review/text", "datasetB.review/text", "distance")

In [66]:
similar_pairs_filtered.orderBy("distance").show(3, truncate=100)

+-------------------------------------------------------------------------------------+--------------------------------------+--------------+--------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------+
|                                                                                Title|                                 Title|       User_id|       User_id|                                                                                         review/text|                                                                                         review/text|          distance|
+-------------------------------------------------------------------------------------+--------------------------------------+--------------+--------------+----------------------------------------------------------------------------------------

In [67]:
# exlude pairs that are too similar (distance < 2.5)
similar_pairs_filtered = similar_pairs_filtered.filter(col("distance") > 2.5)

In [68]:
similar_pairs_filtered.orderBy("distance").show(5, truncate=100)

+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+--------------+--------------+----------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------+------------------+
|                                                                                           Title|                                                                                           Title|       User_id|       User_id|                                                                                         review/text|                                                                                           review/text|          distance|
+------------------------------------------------------------------------------------------------+----